In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import random_split
from pathlib import Path
# Importing all the necessary Libraries

In [2]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307), (0.3081))]
    )

    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform
    )
    return trainset, testset
# defined a function to load the MNIST dataset from the torchvision module and
# defined a transform where we are normalizing the single channel images with mean 0.1307 and standard deviation 0.3081


In [3]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits
# Have used a simple Neural Network which consists of a flattening layer followed by a sequential stack of fully connected layers with ReLU activation.


In [6]:
# have defined a training Neural Network Function and have used help from Pytorch.org
def train_MINST(net,data_dir=None):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(64), shuffle=True
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(64), shuffle=True
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 100 == 99:  # print every 2000 mini-batches
                print( "[%d, %5d] loss: %.3f"% (epoch + 1, i + 1, running_loss / epoch_steps) )
                running_loss = 0.0




In [7]:
#Using Xavier Initalization
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)



In [8]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [9]:
def main():
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    model=Net()
    model.apply(init_weights)
    device="cpu"
    if torch.cuda.is_available():
        device="cuda:0"
        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
    model.to(device)
    epochs=10
    train_MINST(model,data_dir)
    print(test_accuracy(model,device))





if __name__ == "__main__":
    main()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 12933670.69it/s]


Extracting /content/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 346050.57it/s]


Extracting /content/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3195335.46it/s]


Extracting /content/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 11708991.25it/s]
<ipython-input-7-eb079861a3e6>:4: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  torch.nn.init.xavier_uniform(m.weight)


Extracting /content/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/MNIST/raw

[1,   100] loss: 1.182
[1,   200] loss: 0.253
[1,   300] loss: 0.133
[1,   400] loss: 0.091
[1,   500] loss: 0.066
[1,   600] loss: 0.052
[1,   700] loss: 0.040
[2,   100] loss: 0.268
[2,   200] loss: 0.118
[2,   300] loss: 0.077
[2,   400] loss: 0.060
[2,   500] loss: 0.043
[2,   600] loss: 0.038
[2,   700] loss: 0.029
[3,   100] loss: 0.208
[3,   200] loss: 0.097
[3,   300] loss: 0.060
[3,   400] loss: 0.045
[3,   500] loss: 0.034
[3,   600] loss: 0.030
[3,   700] loss: 0.025
[4,   100] loss: 0.151
[4,   200] loss: 0.084
[4,   300] loss: 0.051
[4,   400] loss: 0.038
[4,   500] loss: 0.031
[4,   600] loss: 0.025
[4,   700] loss: 0.020
[5,   100] loss: 0.139
[5,   200] loss: 0.068
[5,   300] loss: 0.043
[5,   400] loss: 0.031
[5,   500] loss: 0.025
[5,   600] loss: 0.021
[5,   700] loss: 0.018
[6,   100] loss: 0.115
[6,   200] loss: 0.056
[6,   300] loss: 0.038
[6,   400] loss: 0.032
[6,   500] los